### A1 Part-1 LLaMA3
#### Hallucinations and RAG

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from sentence_transformers import SentenceTransformer, util
import accelerate
from torch.nn.functional import cosine_similarity
import requests
from bs4 import BeautifulSoup

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

##### LLM Preparation

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
    bnb_8bit_quant_type='nf8',  # Can be 'nf4' or 'fp4'
    bnb_8bit_compute_dtype=torch.bfloat16  # Adjust compute type if needed
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto',
    quantization_config=quantization_config,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def get_model_size(model):
    total_params = sum(p.numel() for p in model.parameters())
    total_size_bytes = total_params * 2
    total_size_gb = total_size_bytes / (1024 ** 3)
    print(f"Model size: {total_size_gb:.2f} GB")

get_model_size(model)

Model size: 14.96 GB


In [ ]:
model.device

device(type='cuda', index=0)

##### RAG Knowledge Base

In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
def split_document_into_passages(document, passage_length=100):
    words = document.split()
    passages = [' '.join(words[i:i+passage_length]) for i in range(0, len(words), passage_length-5)]
    return passages

In [ ]:
def scrape_webpage(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    paragraphs = soup.find_all('p')
    content = "\n".join([para.get_text() for para in paragraphs])
    articles = soup.find_all('article')
    content += "\n".join([para.get_text() for para in articles])
    links = soup.find_all('a')
    content += "\n".join([para.get_text() for para in links])
    
    content = content.replace("\n", " ").strip()
    content_passages = split_document_into_passages(content)
    return content_passages


In [ ]:
urls = [
    'https://www.nobelprize.org/prizes/physics/1921/summary/',
    'https://time.com/5198933/why-stephen-hawking-never-won-nobel/',
    'https://apollo11space.com/apollo-11-space-mission-food/',
    'https://www.merriam-webster.com/wordfinder/classic/begins/all/-1/apt/1',
    'https://en.wikipedia.org/wiki/2021_ICC_Men%27s_T20_World_Cup',
    'https://doodlelearning.com/maths/skills/division/long-division/',
    # 'https://www.idtech.com/blog/easy-division-tricks-for-kids',
    
]

In [ ]:
knowledge_base = []

for url in urls:
    knowledge_base.extend(scrape_webpage(url))

passage_embeddings = embedding_model.encode(knowledge_base, convert_to_tensor=True)

In [ ]:
knowledge_base

["Photo from the Nobel Foundation archive. Prize share: 1/1 To cite this section MLA style: The Nobel Prize in Physics 1921. NobelPrize.org. Nobel Prize Outreach AB 2024. Thu. 5 Sep 2024. <https://www.nobelprize.org/prizes/physics/1921/summary/> Tasked with a mission to manage Alfred Nobel's fortune and has ultimate responsibility for fulfilling the intentions of Nobel's will. For more than a century, these academic institutions have worked independently to select Nobel Prize laureates. Several outreach organisations and activities have been developed to inspire generations and disseminate knowledge about the Nobel Prize. The Nobel Prize The Nobel Prize in Physics 1921 Navigate to: Summary- Albert Einstein",
 'Navigate to: Summary- Albert Einstein Presentation Speech Photo from the Nobel Foundation archive. Albert Einstein Prize share: 1/1 The Nobel Prize in Physics 1921 was awarded to Albert Einstein "for his services to Theoretical Physics, and especially for his discovery of the law

In [ ]:
def retrieve_information(query, top_k=2):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    similarities = cosine_similarity(query_embedding, passage_embeddings)
    
    top_k_indices = torch.topk(similarities, k=top_k).indices
    retrieved_passages = [knowledge_base[idx] for idx in top_k_indices]
    
    return retrieved_passages

##### Response Generation without RAG

In [ ]:
def generate_response(prompt, max_tokens=40):
    messages = [
        {"role": "system", "content": "You are a general purpose chatbot!"},
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=max_tokens,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    del input_ids
    torch.cuda.empty_cache()
    return f'Prompt: {prompt}\n' + f'Response: {tokenizer.decode(response, skip_special_tokens=True)}'

##### Response Generation with RAG

In [ ]:
def generate_response_with_rag(prompt, top_k=4, max_tokens=40, print_retrieved_docs=True):
    retrieved_passages = retrieve_information(prompt, top_k=top_k)
    context = "\n\n".join(retrieved_passages)

    if context:
        if print_retrieved_docs:
            print("Retrieved facts: \n" + '\n'.join([f"{i}. {j}" for i, j in enumerate(context.split("\n\n"))]) + '\n')
        rag_prompt = context + "\n\n" + prompt
    else:
        rag_prompt = prompt

    messages = [
        {"role": "system", "content": "You are a general purpose chatbot!"},
        {"role": "user", "content": rag_prompt},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=max_tokens,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    del input_ids
    torch.cuda.empty_cache()
    return f'Prompt: {prompt}\n' + f'Response: {tokenizer.decode(response, skip_special_tokens=True)}'


##### Example Prompts - Fact Checking

Example 1:   
- Without RAG

In [ ]:
prompt = 'When did Albert Einstein and Stephen Hawking win the Nobel Prize?'
output = generate_response(prompt)
print(output)

Prompt: When did Albert Einstein and Stephen Hawking win the Nobel Prize?
Response: Albert Einstein never won the Nobel Prize in Physics, despite being one of the most influential physicists of the 20th century. He was nominated for the prize several times, but the Nobel Committee did not


- With RAG

In [ ]:
prompt = 'When did Albert Einstein and Stephen Hawking win the Nobel Prize?'
output = generate_response_with_rag(prompt, max_tokens=80)
print(output)

Retrieved facts: 
0. physics, his field’s most famous award eluded Hawking throughout his life. So why did one of science’s most iconic pioneers never win a Nobel Prize in Physics? The answer — unlike quantum mechanics — is relatively straightforward. Theoretical scientific discoveries have to be confirmed by observational data before there’s a possibility of winning a Nobel. And it’s somewhat difficult to
1. science’s most iconic pioneers never win a Nobel Prize in Physics?The answer — unlike quantum mechanics — is relatively straightforward.Theoretical scientific discoveries have to be confirmed by observational data before there’s a possibility of winning a Nobel. And it’s somewhat difficult to observe a black hole.It takes decades to build the scientific equipment to test theoretical discoveries; to put this
2. The issue is, the technology needed to observe this radiation will take years and cost millions before Hawking’s theory can ever be verified.Hawking never won a Nobel, but a

Example 2:   
- Without RAG

In [ ]:
prompt = 'Neil Armstrong did not eat on the Apollo 11 module. Yes or no?'
output = generate_response(prompt)
print(output)

Prompt: Neil Armstrong did not eat on the Apollo 11 module. Yes or no?
Response: Yes.


- With RAG

In [ ]:
prompt = 'Neil Armstrong did not eat on the Apollo 11 module. Yes or no?'
output = generate_response_with_rag(prompt)
print(output)

Retrieved facts: 
0. that Neil Armstrong’s favorite meal was spaghetti with meat sauce, scalloped potatoes, fruitcake cubes, and grape punch, a spread that certainly sounds Retro Recipe ready. The first food eaten on the Moon was a communion wafer. When Apollo 11‘s Eagle lunar module landed on the Moon on July 20, 1969, the two astronauts, Buzz Aldrin and Neil Armstrong, had to
1. spaghetti with meat sauce, scalloped potatoes, fruitcake cubes, and grape punch, a spread that certainly sounds Retro Recipe ready. The first food that was eaten on the Moon The first food eaten on the Moon was a communion wafer. When Apollo 11‘s Eagle lunar module landed on the Moon on July 20, 1969, the two astronauts, Buzz Aldrin and Neil
2. Apollo rehydratable food packaging. Buzz Aldrin later enjoyed the shrimp cocktail, explaining, “The shrimp were chosen one by one to be sure they would be tiny enough to squeeze out of the food packet, and they were delicious!” From First Man: The Life of Neil A. Armst

Example 3:   
- Without RAG

In [ ]:
prompt = 'Tell me about "Aptomania".'
output = generate_response(prompt)
print(output)

Prompt: Tell me about "Aptomania".
Response: Aptomania! That's a fascinating topic. Aptomania refers to the widespread and intense enthusiasm for aptitude tests, particularly the popular ones like the Myers-Briggs Type Indicator (MB


- With RAG

In [ ]:
prompt = 'Tell me about "Aptomania".'
output = generate_response_with_rag(prompt, max_tokens=60)
print(output)

Retrieved facts: 
0. aptronyms apts aptyalism aptychi aptychus Can you solve 4 words at once? Play Play Can you solve 4 words at once? Play Play Word of the Day ensconce See Definitions and Examples Merriam Webster Help About Us Advertising Info Contact Us Diversity Privacy Policy Terms of Use Facebook Twitter YouTube Instagram
1. Join MWU Books Merch Log In My Words Recents Account Log Out Word Finder apt Aptal Aptals aptamer Aptenodytes apter aptera apterae apteral apteran apteria apterial apteries apterium apterous aptery Apteryges apterygial Apterygiformes Apterygogenea Apterygota apterygote apterygotous apteryla apterylae apteryx apteryxes aptest aptha apthae Aptian Aptiana apting aptitude aptitudes aptitudinal aptitudinally aptly aptness aptnesses aptronym aptronyms apts aptyalism aptychi aptychus
2. veggies. Credit: Bettmann/Getty Are you keen on exploring the ins and outs of lunar missions? Our Complete Guide to the Apollo Program is the ultimate compendium. Find out what it to

##### Example Prompts - Self-consistency

Example 1:   
- Without RAG

In [ ]:
prompt = '256789/5 = ? Just give the answer.'
output1 = generate_response(prompt)
print(output1)

prompt = 'If 256789 is divided by 5, what will be the result? Just give the answer.'
output2 = generate_response(prompt)
print(output2)

prompt = 'What is the result of 256789 divided by 5? Just give the answer.'
output3 = generate_response(prompt)
print(output3)

Prompt: 256789/5 = ? Just give the answer.
Response: 51.36
Prompt: If 256789 is divided by 5, what will be the result? Just give the answer.
Response: 51.4
Prompt: What is the result of 256789 divided by 5? Just give the answer.
Response: 51375.8


- With RAG

In [ ]:
prompt = '256789/5 = ? Just give the answer.'
output1 = generate_response_with_rag(prompt, top_k=4)
print(output1)

prompt = 'If 256789 is divided by 5, what will be the result? Just give the answer.'
output2 = generate_response_with_rag(prompt, top_k=4, print_retrieved_docs=False)
print(output2)

prompt = 'What is the result of 256789 divided by 5? Just give the answer.'
output3 = generate_response_with_rag(prompt, top_k=4, print_retrieved_docs=False)
print(output3)

# 51357.8

Retrieved facts: 
0. 225÷5 Make sure the divisor is written first, and separate it from the dividend by either a right parentheses [)] or a backslash [/]. Then, draw an overhead bar above the dividend that connects to the parentheses or backslash you just drew. This will help you understand the difference between divisor and dividend. Now, let’s take a look at the first number from the left in our divisor. Since 2 is less than 5, the dividend, we know we can’t do any division here. So, we write a 0 as our first number in the quotient and multiply it by
1. quotient and multiply it by our 5, giving us a product of 0. We then need to subtract this 0 from the first digit of the dividend, 2. Now we can drag the 2 in the tens place down and place it next to the difference we calculated in step 2. This gives us a value of 22. 22 isn’t divisible by 5, but we know 5 x 4 = 20, or that 5 fits into 20 four times. That means we can write a four in the quotient to the right of our 0. We then write t

Example 2:   
- Without RAG

In [ ]:
prompt = 'If Ram has 14 apples and he gives 3 to Shyam and receives 4 from Ramesh, how many apples does Ram have now? Give number only.'
output1 = generate_response(prompt)
print(output1)

prompt = 'Out of a total of 20 apples 14 are given to Ram, and the remaining to Ramesh. Shyam receives 3 apples from Ram, who receives 4 from Ramesh. How many apples does Ram have now? Just give the number.'
output2 = generate_response(prompt)
print(output2)

Prompt: If Ram has 14 apples and he gives 3 to Shyam and receives 4 from Ramesh, how many apples does Ram have now? Give number only.
Response: 11
Prompt: Out of a total of 20 apples 14 are given to Ram, and the remaining to Ramesh. Shyam receives 3 apples from Ram, who receives 4 from Ramesh. How many apples does Ram have now? Just give the number.
Response: 8


- With RAG

In [ ]:
prompt = 'If Ram has 14 apples and he gives 3 to Shyam and receives 4 from Ramesh, how many apples does Ram have now?'
output1 = generate_response(prompt, max_tokens=200)
print(output1)

prompt = 'Out of a total of 20 apples 14 are given to Ram, and the remaining to Ramesh. Shyam receives 3 apples from Ram, who receives 4 from Ramesh. How many apples does Ram have now?'
output2 = generate_response(prompt, max_tokens=200, print_retrieved_docs=False)
print(output2)

Retrieved facts: 
0. 2 x 4 to get 8, place the product directly below the 9, and draw a line beneath it: Subtract 9 â 8 to get 1. (Note: After you subtract, the result should be less than the divisor (in this problem, the divisor is 4). Then bring down the next number (5) to make the new number 15. These steps are one complete cycle. To complete the problem, you just need to repeat them. Now ask how many times 4 goes into 15 â that is, whatâs 15 / 4? The answer is 3 (with a little left over). So
1. problem, you just need to repeat them. Now ask how many times 4 goes into 15 â that is, whatâs 15 / 4? The answer is 3 (with a little left over). So write the 3 above the 5, and then multiply 3 x 4 to get 12. Write the product under 15. Subtract 15 â 12 to get 3. Then bring down the next number (6) to make the new number 36. Another cycle is complete, so begin the next cycle by asking how many times 4 goes into 36 â that is, whatâs 36 / 4? The answer
2. a little left over). S

Example 3:   
- Without RAG

In [ ]:
prompt = 'Who won the 2020 Mens Cricket World Cup?'
output1 = generate_response(prompt)
print(output1)

prompt = 'Which team won the Mens Cricket World Cup in 2020?'
output2 = generate_response(prompt, max_tokens=50)
print(output2)

prompt = 'Did England win the 2020 Cricket World Cup? Yes or no?'
output3 = generate_response(prompt)
print(output3)

Prompt: Who won the 2020 Mens Cricket World Cup?
Response: The 2019 ICC Men's Cricket World Cup was won by the England national cricket team. They defeated New Zealand in the final at Lord's on July 14, 2019.
Prompt: Which team won the Mens Cricket World Cup in 2020?
Response: The ICC Men's Cricket World Cup 2019 was won by the England national cricket team, who defeated New Zealand in the final at Lord's on July 14, 2019. There was no Cricket World Cup held in 2020,
Prompt: Did England win the 2020 Cricket World Cup? Yes or no?
Response: Yes.


- With RAG

In [ ]:
prompt = 'Who won the 2020 Mens Cricket World Cup?'
output1 = generate_response(prompt)
print(output1)

prompt = 'Which team won the Mens Cricket World Cup in 2020?'
output2 = generate_response(prompt, max_tokens=50, print_retrieved_docs=False)
print(output2)

prompt = 'Did England win the 2020 Cricket World Cup? Yes or no?'
output3 = generate_response(prompt, print_retrieved_docs=False)
print(output3)

Retrieved facts: 
0. Twenty20 final Cricket World Cup ODI ICC World Test Championship [16] [17] COVID-19 pandemic [18] [19] [20] [21] [22] [23] Cricket Australia Women's Cricket World Cup [24] Simon Birmingham [25] [26] [27] 2022 [28] [29] Geoff Allardice [30] Board of Control for Cricket in India [31] pandemic in India [32] [33] [34] [35] [36] Test-playing nations [37] Cyclone Shaheen Oman Cricket [38] edit 2019 ICC Men's T20 World Cup Qualifier ICC Africa Namibia South Africa Americas West Indies Asia Afghanistan Bangladesh India Oman Pakistan Sri Lanka East Asia-Pacific Australia New Zealand Papua New Guinea Europe England Ireland Netherlands Scotland Full
1. of the Tournament" edit Series home v t e 2021 ICC Men's T20 World Cup First Round Group A Group B Super 12 Group 1 Group 2 Knockout Stage Final Qualifier Africa Americas Asia East Asia-Pacific Europe Squads Statistics Umpires and Officials Venues Warm-up matches v t e ICC Men's T20 World Cup International Cricket Council South

Prompt: Who won the 2020 Mens Cricket World Cup?
Response: There was no 2020 Men's Cricket World Cup. The 2019 ICC Cricket World Cup was held in England and Wales, and the winner was the England national team. They defeated New Zealand
Prompt: Which team won the Mens Cricket World Cup in 2020?
Response: There was no Men's Cricket World Cup held in 2020. The 2019 ICC Men's Cricket World Cup was held in England and Wales, and the winner was the England national cricket team. The 2021 ICC Men's T20
Prompt: Did England win the 2020 Cricket World Cup? Yes or no?
Response: No.
